In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Extract Music Files
zip_path = '/content/drive/My Drive/AAI511_ML/midi_classic_music.zip'
extract_path = '/content/midi_classic_music/'

# Unzip the file.
!unzip -q -n "{zip_path}" -d "{extract_path}"

import shutil

composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']

for item in os.listdir(extract_path):
    item_path = os.path.join(extract_path, item)
    if os.path.isfile(item_path) and (item.endswith('.mid') or item.endswith('.MID')):
        os.remove(item_path)
        # print(f"Remove file at root: {item}")

midi_classic_path = extract_path + 'midiclassics'

for item in os.listdir(midi_classic_path):
    item_path = os.path.join(midi_classic_path, item)
    if os.path.isfile(item_path) and (item.endswith('.mid') or item.endswith('.MID')):
        os.remove(item_path)
        # print(f"Remove file at midi_classic_path: {item}")

# remove rest of non-composer files.
for root, dirs, files in os.walk(extract_path, topdown=False):
    for dir_name in dirs:
        if dir_name in composers:
            continue  # Keep this directory

        full_path = os.path.join(root, dir_name)
        contains_desired = False

        for subroot, subdirs, subfiles in os.walk(full_path):
            if any(composer in subdirs for composer in composers):
                contains_desired = True
                break

        if not contains_desired:
            shutil.rmtree(full_path)
            # print(f"Removed: {full_path}") for debugging

Mounted at /content/drive


In [2]:
!pip install pypianoroll

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.7 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=edc05a2c1306acc33a17136a2feca6395dd82ac63a45a7f4a9349f5607327824
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty-midi


In [3]:
#
# A script for preprocessing MIDI files using the Piano Roll method.
#
# Major Improvements from the previous token-based approach:
# 1. Image-like Representation: Converts MIDI files into 2D piano roll arrays
#    (time vs. pitch), which is an ideal format for Convolutional Neural Networks (CNNs).
# 2. Simplified Pipeline: Eliminates the need for complex vocabulary building, tokenizing,
#    and sequence generation steps. The data is already numerical.
# 3. Direct Data Handling: Creates final training and validation sets directly using
#    a standard train-test split, ready for model training.
#

import os
from pathlib import Path
from multiprocessing import Pool, cpu_count
import numpy as np
import pypianoroll
from sklearn.model_selection import train_test_split

# --- Configuration ---
MIDI_ROOT_DIR = '/content/midi_classic_music/midiclassics/'
# The fixed number of timesteps for each piano roll. Shorter songs are padded, longer ones are truncated.
FIXED_TIMESTEPS = 5000
# The number of pitches in the MIDI standard.
N_PITCHES = 128
# Filename for the final preprocessed data
OUTPUT_NPZ_FILE = 'preprocessed_pianoroll_data.npz'

# --- Helper Functions ---

def get_songs_by_composer(root_dir):
    """Finds all MIDI files and groups them by composer from the directory structure."""
    songs_dict = {}
    root_path = Path(root_dir)
    for composer_path in root_path.iterdir():
        if composer_path.is_dir():
            composer_name = composer_path.name
            # Use .glob() to find all .mid files, including case variations like .MID
            songs_dict[composer_name] = list(composer_path.glob('**/*.[mM][iI][dD]'))
    return songs_dict

def midi_to_pianoroll(file_path):
    """
    Parses a MIDI file and converts it into a standardized piano roll numpy array.
    """
    try:
        # Load the MIDI file using pypianoroll
        multitrack = pypianoroll.read(str(file_path))

        # Merge all tracks into a single piano roll
        multitrack.blend('sum')
        pianoroll = multitrack.tracks[0].pianoroll

        # Binarize the roll: 1 if a note is played, 0 otherwise
        pianoroll[pianoroll > 0] = 1

        # Standardize the length (timesteps) of the piano roll
        n_timesteps = pianoroll.shape[0]
        if n_timesteps < FIXED_TIMESTEPS:
            # Pad with zeros if the song is too short
            padding = np.zeros((FIXED_TIMESTEPS - n_timesteps, N_PITCHES))
            standardized_pianoroll = np.vstack([pianoroll, padding])
        else:
            # Truncate if the song is too long
            standardized_pianoroll = pianoroll[:FIXED_TIMESTEPS, :]

        return standardized_pianoroll

    except Exception as e:
        # Print an error if a file fails to parse and return None
        print(f" - Could not parse {os.path.basename(file_path)}: {e}")
        return None

# --- Main Execution Block ---
if __name__ == '__main__':
    # Step 1: Discover files and create a map of composers to file paths
    # This logic remains the same, as it correctly finds the files.
    print("Step 1: Finding MIDI files and mapping composers...")
    songs_dict = get_songs_by_composer(MIDI_ROOT_DIR)
    all_composers = sorted(songs_dict.keys())
    composer_map = {name: i for i, name in enumerate(all_composers)}
    num_classes = len(all_composers)
    print(f"Found {num_classes} composers: {', '.join(all_composers)}")

    # Create a flat list of all file paths and a corresponding list of their integer labels
    all_files = []
    all_labels = []
    for composer, files in songs_dict.items():
        composer_id = composer_map[composer]
        for file in files:
            all_files.append(file)
            all_labels.append(composer_id)

    # Step 2: Split files into training and validation sets BEFORE processing
    # This is a crucial step to prevent data leakage. We split the file list, not the data arrays.
    print("\nStep 2: Splitting files into training and validation sets...")
    train_files, val_files, train_labels, val_labels = train_test_split(
        all_files,
        all_labels,
        test_size=0.2,
        random_state=42,
        stratify=all_labels  # Ensures composer distribution is similar in train/val sets
    )
    print(f"Training files: {len(train_files)}, Validation files: {len(val_files)}")

    # Step 3: Process MIDI files into piano rolls in parallel
    print(f"\nStep 3: Processing MIDI files with {cpu_count()} cores...")

    # Process training files
    print("Processing training files...")
    with Pool() as pool:
        X_train_list = pool.map(midi_to_pianoroll, train_files)

    # Process validation files
    print("Processing validation files...")
    with Pool() as pool:
        X_val_list = pool.map(midi_to_pianoroll, val_files)

    # Step 4: Final data cleaning and assembly
    # This step replaces the old vocabulary building and sequence creation.
    print("\nStep 4: Assembling final datasets...")

    # Filter out any files that failed to parse (returned None) and their corresponding labels
    y_train = np.array([label for i, label in enumerate(train_labels) if X_train_list[i] is not None])
    X_train = np.array([roll for roll in X_train_list if roll is not None])

    y_val = np.array([label for i, label in enumerate(val_labels) if X_val_list[i] is not None])
    X_val = np.array([roll for roll in X_val_list if roll is not None])

    print(f"Training set shape:   X={X_train.shape}, y={y_train.shape}")
    print(f"Validation set shape: X={X_val.shape}, y={y_val.shape}")

    print("Preprocessing complete. Data is ready for model training.")

Step 1: Finding MIDI files and mapping composers...
Found 4 composers: Bach, Beethoven, Chopin, Mozart

Step 2: Splitting files into training and validation sets...
Training files: 392, Validation files: 99

Step 3: Processing MIDI files with 2 cores...
Processing training files...


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


 - Could not parse Anhang 14-3.mid: Could not decode key with 3 flats and mode 255


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Processing validation files...


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pypianoroll/inputs.py:225: RuntimeWarning: divide by zero encountered in divide
  bpm = 60.0 / np.diff(beat_times_one_more)



Step 4: Assembling final datasets...
Training set shape:   X=(391, 5000, 128), y=(391,)
Validation set shape: X=(99, 5000, 128), y=(99,)
Preprocessing complete. Data is ready for model training.


In [4]:
# we had to go through several iterations to get this right!
save_path = '/content/drive/MyDrive/AAI511_ML/preprocessed_composer_data_vpiano_roll.npz'

print(f"\nSaving pre-processed data to {save_path}...")

np.savez_compressed(
        save_path,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val
    )

# old method
# np.savez_compressed(
#     save_path,
#     network_input=network_input,
#     network_output=network_output,
#     n_vocab=np.array([n_vocab]), # Save n_vocab as a numpy array
#     composer_map=composer_map,   # Save the composer mapping dictionary
#     note_to_int=note_to_int      # Save the note mapping dictionary
# )

print("Data saved")


Saving pre-processed data to /content/drive/MyDrive/AAI511_ML/preprocessed_composer_data_vpiano_roll.npz...
Data saved


In [5]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the preprocessed data
print(f"Loading preprocessed data from: {save_path}")
with np.load(save_path, allow_pickle=True) as data:
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']


def create_optimized_dataset(features, labels, batch_size=64, is_training=True):
    """Create an optimized dataset."""
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    if is_training:
        dataset = dataset.shuffle(len(features), reshuffle_each_iteration=True)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the datasets
BATCH_SIZE = 128
train_dataset = create_optimized_dataset(X_train, y_train, batch_size=BATCH_SIZE)
validation_dataset = create_optimized_dataset(X_val, y_val, batch_size=BATCH_SIZE, is_training=False)

# update xtrain/ytrain

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)

print("\nTensorFlow datasets created.")

Loading preprocessed data from: /content/drive/MyDrive/AAI511_ML/preprocessed_composer_data_vpiano_roll.npz

TensorFlow datasets created.


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def create_cnn_model(input_shape, num_classes):
    """
    Creates a 2D CNN model for piano roll classification.
    """
    # The input shape will now include the channel dimension, e.g., (5000, 128, 1)
    inputs = Input(shape=input_shape)

    # --- Convolutional Block 1 ---
    # The first layer now takes 'inputs' directly
    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # ... the rest of the function remains the same ...

    # --- Convolutional Block 2 ---
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # --- Convolutional Block 3 ---
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # --- Flatten and Classify ---
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Get model parameters from the data
input_shape = X_train.shape[1:]
num_classes = 4

# Build and compile the model
print("\nBuilding and compiling the model...")
model = create_cnn_model(input_shape=input_shape, num_classes=4) # 4 is the num composers

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # adjusted learning rate after some attempts
    loss='sparse_categorical_crossentropy', # Use this for integer labels
    metrics=['accuracy']
)

model.summary()

# Add a callback to stop training early if performance plateaus
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=5,          # Stop after 5 epochs with no improvement
    restore_best_weights=True # Restore the best model weights found
)

print("\nStarting model training...")
history = model.fit(
    train_dataset,
    epochs=40, # Set a high number, early stopping will find the best epoch
    validation_data=validation_dataset,
    callbacks=[early_stopping]#,
    # verbose=0 # to keep code clean
)
print("\nModel training complete")


Building and compiling the model...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5000, 128, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 4998, 126, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2499, 63, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2499, 63, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2497, 61, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1248, 30, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1248, 30, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1246, 28, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 623, 14, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 623, 14, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 558208)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    71,450,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,507,012 (272.78 MB)

 Trainable params: 71,507,012 (272.78 MB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/40


UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyk